## Preliminaries
### Import statements

In [1]:
import pandas as pd
import spacy
import os
import re

### Files to parse

In [2]:
seneca_files = [
    'Sen_Agamemnon.csv',	
    'Sen_Hercules_Oetaeus.csv',
    'Sen_Hercules_furens.csv',	
    'Sen_Medea.csv',		
    'Sen_Oedipus.csv',		
    'Sen_Phaedra.csv',		
    'Sen_Phoenissae.csv',	
    'Sen_Thyestes.csv',	
    'Sen_Troades.csv',
]

### Load spaCy model

In [3]:
nlp = spacy.load('la_core_web_lg')

## Parse the texts

### Load and parse the texts

The CSV files in `data/` contain line-by-line alignments of Perseus' text and ground-truth scansion from [Fedchin, Burns, Chaudhuri, and Dexter (2022) "Senecan Trimeter and Humanist Tragedy"](https://github.com/QuantitativeCriticismLab/AJP-2022-Senecan-Trimeter). Here, we load the files, parse the text with spaCy, and create a Pandas DataFrame with one row per token.

In [4]:
tables = []
offset = 100000

for file in seneca_files:
    path = os.path.join('data', 'seneca', file)
    print(path)
    df = pd.read_csv(path)
    df = df.loc[~df.elision.isna()]
    df = df.loc[~df.speech.isna()]
    df.perseus_n = df.perseus_n.str.replace(r'[\[\]\']', '', regex=True)
    df.speaker = df.speaker.str.replace(r'[\[\]\'#]', '', regex=True)
    df.elision = df.elision.astype(int)
    df.speech = df.speech.astype(int) + offset
    df.anceps_n = df.anceps_n.astype(int)
    df['tokens'] = df.perseus_text.apply(lambda s: [tok for tok in nlp(s)])
    df['file'] = file[:-4]
    df['line_id'] = df['file'] + ':' + df['perseus_n']
    tables.append(df)
    offset += 10000

df = pd.concat(tables)
df['line_id'] = pd.Categorical(df['line_id'], categories=pd.unique(df['line_id']), ordered=True)

data/seneca/Sen_Agamemnon.csv
data/seneca/Sen_Hercules_Oetaeus.csv
data/seneca/Sen_Hercules_furens.csv
data/seneca/Sen_Medea.csv
data/seneca/Sen_Oedipus.csv
data/seneca/Sen_Phaedra.csv
data/seneca/Sen_Phoenissae.csv
data/seneca/Sen_Thyestes.csv
data/seneca/Sen_Troades.csv


### Helper function to extract elisions from the scansion

In [5]:
def normalize(s):
    return re.sub(r'[^a-z]', '', s.lower()).replace('jv', 'iu')

def getElided(df):
    '''extract elided tokens from a line-array table'''
    rows = []
    row_count = 0

    for row in df.itertuples():
        elided = [False] * len(row.tokens)
        row_count += 1
        
        if '(' in row.anceps_text:
            anceps = row.anceps_text.split()
            no_punct = [tok for tok in row.tokens if tok.pos_ != 'PUNCT']
            
            if len(no_punct) == len(anceps):
                for anc, tok in zip(anceps, no_punct):
                    if '(' in anc:
                        idx = row.tokens.index(tok)
                        elided[idx] = True
            else:
                for anc in anceps:
                    if '(' in anc:
                        normalized_anc = normalize(anc)
                        normalized_toks = [normalize(tok.text) for tok in row.tokens]
                        count = normalized_toks.count(normalized_anc)
                        if count == 1:
                            idx = normalized_toks.index(normalized_anc)
                        elif anc.endswith('qv(e)') and normalized_toks.count('que') == 1:
                            idx = normalized_toks.index('que')
                        else:
                            print(f'[{row_count}]\t' + ' '.join([f'{i}.{tok.text}' for i, tok in enumerate(row.tokens)]))
                            idx = int(input(f'Which is {anc}? '))
                        elided[idx] = True
        rows.append(elided)
    return rows

### Calculate elisions

In [7]:
df['elided'] = getElided(df)

[65]	0.te 1.cum 2.ipsa 3.nunc 4.evolve 5.femineos 6.dolos 7.,


Which is te*c(u)m?  1


[170]	0.iacens 1.que 2.vultu 3.languido 4.optutus 5.stupet 6.?


Which is la_ngvi^d(o)?  3


[225]	0.Exilia 1.mihi 2.sunt 3.haud 4.nova 5., 6.assuevi 7.malis 8..


...

Which is a_tqv(e)?  6


[6483]	0.similis 1.que 2.maesto 3., 4.squalida 5.obtectus 6.coma 7..


Which is sqva_li^d(a)?  4


[6565]	0.rerum 1.aestimator 2.: 3.si 4.tamen 5.te 6.cum 7.exigas 8.,


Which is te*c(u)m?  6


### Write output

In [8]:
df.to_csv(os.path.join('data','seneca_scanned.csv'), index=False)
df

,perseus_n,perseus_text,comp,anceps_n,anceps_text,speaker,elision,speech,tokens,file,line_id,elided
0,1,Opaca linquens Ditis inferni loca,0.900000,0,o*pa_ca^ li_nqve*ns di_ti^s i_nfe*rni_ lo^ca*,thyestis,0,100001,"[Opaca, linquens, Ditis, inferni, loca]",Sen_Agamemnon,Sen_Agamemnon:1,"[False, False, False, False, False]"
1,2,"adsum profundo Tartari emissus specu,",1.000000,1,a*dsu_m pro^fu_ndo* ta_rta^r(i) e_mi*ssu_s sp...,thyestis,1,100001,"[adsum, profundo, Tartari, emissus, specu, ,]",Sen_Agamemnon,Sen_Agamemnon:2,"[False, False, True, False, False, False]"
2,3,incertus utras oderim sedes magis:,1.000000,2,i*nce_rtu^s u_tra*s o_de^ri_m se*de_s ma^gi*s,thyestis,0,100001,"[incertus, utras, oderim, sedes, magis, :]",Sen_Agamemnon,Sen_Agamemnon:3,"[False, False, False, False, False, False]"
3,4,"fugio Thyestes inferos, superos fugo.",1.000000,3,fu^gi^o_ thy^e_ste*s i_nfe^ro_s su^pe^ro_s fu...,thyestis,0,100001,"[fugio, Thyestes, inferos, ,, superos, fugo, .]",Sen_Agamemnon,Sen_Agamemnon:4,"[False, False, False, False, False, False, False]"
4,5,en horret animus et pavor membra excutit:,1.000000,4,e*n ho_rre^t a^ni^mu*s e_t pa^vo_r me*mbr(a) ...,thyestis,1,100001,"[en, horret, animus, et, pavor, membra, excuti...",Sen_Agamemnon,Sen_Agamemnon:5,"[False, False, False, False, False, True, Fals..."
...,...,...,...,...,...,...,...,...,...,...,...,...
1172,1173,"ubique properas, saeva: me solam times",0.909091,912,u*bi_qve^ pro^pe^ra*s s[ae]va^ me_ so*la_m ti...,hecuba,0,180051,"[ubique, properas, ,, saeva, :, me, solam, times]",Sen_Troades,Sen_Troades:1173,"[False, False, False, False, False, False, Fal..."
1173,1174,"vitasque, gladios inter ac tela et faces",0.916667,913,vi*ta_sqve^ gla^di^o*s i_nte^r a_c te*l(a) e_...,hecuba,1,180051,"[vitas, que, ,, gladios, inter, ac, tela, et, ...",Sen_Troades,Sen_Troades:1174,"[False, False, False, False, False, False, Tru..."
1174,1175,"quaesita tota nocte, cupientem fugis. ,",0.937500,914,qv[ae]si_ta^ to_ta* no_cte^ cu^pi^e*nte_m fu^...,hecuba,0,180051,"[quaesita, tota, nocte, ,, cupientem, fugis, ....",Sen_Troades,Sen_Troades:1175,"[False, False, False, False, False, False, Fal..."
1175,1176,"non hostis aut ruina, non ignis meos",1.000000,915,no*n ho_sti^s [au]t ru*i_na^ no_n i*gni_s me^o*s,hecuba,0,180051,"[non, hostis, aut, ruina, ,, non, ignis, meos]",Sen_Troades,Sen_Troades:1176,"[False, False, False, False, False, False, Fal..."


### Custom function to define local window for repetitions

We treat the words before and after as a "context"; if the present word is also found among them, then it's counted as a repetition.

In [9]:
def getContext(token_table):
    df = (token_table
        .assign(lemma = token_table.lemma.str.lower())
        .groupby('line_id', as_index = False)
        .agg(line_id = ('line_id', 'first'), lemmas = ('lemma', list))
    )

    r_context = (
        pd.concat(
            pd.DataFrame(dict(
                line_id = df.iloc[:-i].line_id.values,
                lemmas = df.iloc[i:].lemmas.values,
            )) for i in range(1, 3))
        .groupby('line_id', as_index=False)
        .agg(r_context=('lemmas', lambda lems: sum(lems, [])))
    )

    l_context = (
        pd.concat(
            pd.DataFrame(dict(
                line_id = df.iloc[i:].line_id.values,
                lemmas = df.iloc[:-i].lemmas.values,
            )) for i in range(1, 3))
        .groupby('line_id', as_index=False)
        .agg(l_context=('lemmas', lambda lems: sum(lems, [])))
    )

    context = pd.merge(l_context, r_context, how='outer', on='line_id')
    context = pd.merge(df, context, on='line_id')
    context['context'] = context['l_context'] + context['lemmas'] + context['r_context']
    context = context.drop(columns=['l_context', 'lemmas', 'r_context'])

    return context

In [12]:
tables = []
for label, group in df.groupby('speech', observed=False):
    print(label)
    token_table = group.explode(['tokens', 'elided']).rename(columns={'tokens':'token'})
    fulltext = ' '.join(group.perseus_text)
    token_table['token'] = [tok for tok in nlp(fulltext)]
    token_table['lemma'] = [tok.lemma_ for tok in token_table.token]
    context = getContext(token_table)
    reps = pd.merge(token_table, context, how='left', on='line_id')[['lemma', 'context']]
    reps['lemma'] = reps['lemma'].str.lower()
    token_table['reps'] = reps.apply(lambda row: row['context'].count(row['lemma']), axis=1).values

    tables.append(token_table)

100001


/var/folders/2x/rfhzgyjn2zq2wlzk_kn4th680000gn/T/ipykernel_9315/2342058334.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df = (token_table
/var/folders/2x/rfhzgyjn2zq2wlzk_kn4th680000gn/T/ipykernel_9315/2342058334.py:9: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  pd.concat(
/var/folders/2x/rfhzgyjn2zq2wlzk_kn4th680000gn/T/ipykernel_9315/2342058334.py:19: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  pd.concat(
/v

100002
100003


/var/folders/2x/rfhzgyjn2zq2wlzk_kn4th680000gn/T/ipykernel_9315/2342058334.py:9: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  pd.concat(
/var/folders/2x/rfhzgyjn2zq2wlzk_kn4th680000gn/T/ipykernel_9315/2342058334.py:19: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  pd.concat(
/var/folders/2x/rfhzgyjn2zq2wlzk_kn4th680000gn/T/ipykernel_9315/2342058334.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df = (token_table
/v

100004
100005


...

/var/folders/2x/rfhzgyjn2zq2wlzk_kn4th680000gn/T/ipykernel_9315/2342058334.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df = (token_table
/var/folders/2x/rfhzgyjn2zq2wlzk_kn4th680000gn/T/ipykernel_9315/2342058334.py:9: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  pd.concat(
/var/folders/2x/rfhzgyjn2zq2wlzk_kn4th680000gn/T/ipykernel_9315/2342058334.py:19: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  pd.concat(


180050


/var/folders/2x/rfhzgyjn2zq2wlzk_kn4th680000gn/T/ipykernel_9315/2342058334.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df = (token_table
/var/folders/2x/rfhzgyjn2zq2wlzk_kn4th680000gn/T/ipykernel_9315/2342058334.py:9: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  pd.concat(
/var/folders/2x/rfhzgyjn2zq2wlzk_kn4th680000gn/T/ipykernel_9315/2342058334.py:19: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  pd.concat(
/v

180051


/var/folders/2x/rfhzgyjn2zq2wlzk_kn4th680000gn/T/ipykernel_9315/2342058334.py:9: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  pd.concat(
/var/folders/2x/rfhzgyjn2zq2wlzk_kn4th680000gn/T/ipykernel_9315/2342058334.py:19: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  pd.concat(


In [13]:
token_table = pd.concat(tables)

token_table['upos'] = [tok.pos_ for tok in token_table.token]
token_table['morph'] = [tok.morph.to_dict() for tok in token_table.token]
token_table['mood'] = [morph.get('Mood') for morph in token_table.morph]
token_table['tense'] = [morph.get('Tense') for morph in token_table.morph]
token_table['voice'] = [morph.get('Voice') for morph in token_table.morph]
token_table['person'] = [morph.get('Person') for morph in token_table.morph]
token_table['number'] = [morph.get('Number') for morph in token_table.morph]
token_table['case'] = [morph.get('Case') for morph in token_table.morph]
token_table['gender'] = [morph.get('Gender') for morph in token_table.morph]
token_table = token_table.drop(columns=['morph', 'perseus_text', 'anceps_text', 'anceps_n', 'comp'])
token_table = token_table.loc[token_table.upos != 'PUNCT']
token_table.to_csv(os.path.join('data', 'seneca_tokens.csv'), index=False)